# FAST-ODD - Uncertainty analysis

The purpose of this notebook is to run uncertainty analyses in order to evaluate the impacts of models' and input parameters uncertainty on the performance.

## 1. Setting up and analyzing the initial problem

In a first place, we define in the configuration file the UAV models that will be used to run the analyses. Here, no optimization of the design is carried out, i.e. we start from an existing design and analyse its performance on a given mission.

In [1]:
import os.path as pth
import openmdao.api as om
import logging
import shutil
import fastoad.api as oad
from time import time
from openmdao_drivers.cmaes_driver import CMAESDriver
from nrel_openmdao_extensions.nlopt_driver import NLoptDriver
import cma
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"]=16,8
plt.rcParams.update({'font.size': 13})
#from openmdao_extensions.openturns_doe_driver import OpenturnsDOEDriver
#from openmdao_extensions.openturns_doe_driver import OPENTURNS_NOT_INSTALLED
#import openturns as ot

DATA_FOLDER_PATH = 'data'
WORK_FOLDER_PATH = 'workdir'

# For having log messages display on screen
#logging.basicConfig(level=logging.INFO, format='%(levelname)-8s: %(message)s')

# For using all screen width
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

D:\Softwares\Anaconda3\envs\OAD\lib\site-packages\pyoptsparse\pyOpt_MPI.py:68: UserWarning: mpi4py could not be imported. mpi4py is required to use the parallel gradient analysis and parallel objective analysis for non-gradient based optimizers. Continuing using a dummy MPI module from pyOptSparse.
  warnings.warn(warn)


Let's start by retrieving the parameters from an existing UAV design. This could be obtained by running an MDO (see other notebooks). We also define the configuration of the problem, i.e. the models that will be used for further performance and uncertainty analysis.

In [2]:
CONFIGURATION_FILE = pth.join(DATA_FOLDER_PATH, 'multirotor_performance_DoE.yaml')
SOURCE_FILE  = pth.join(WORK_FOLDER_PATH, "problem_outputs_continuous.xml")
INPUT_FILE = oad.generate_inputs(CONFIGURATION_FILE, SOURCE_FILE, overwrite=True)

FileNotFoundError: [Errno 2] No such file or directory: 'D:\\THESE\\Tools\\FAST-ODD\\data\\multirotor_performance_DoE.yaml'

In [ ]:
N2_FILE = pth.join(WORK_FOLDER_PATH, 'n2.html')
oad.write_n2(CONFIGURATION_FILE, N2_FILE, overwrite=True)
from IPython.display import IFrame
IFrame(src=N2_FILE, width='100%', height='500px')

We can now modify parameters from the retrieved design. For instance, one may change the mission definition to check for the performance, or change a model's parameter to analyse its effect on the performance.

In [ ]:
oad.variable_viewer(INPUT_FILE)

To obtain the performance of the UAV design, we run the problem onced. No optimization is carried out, only a straightforward calculation of the models.

In [ ]:
perfo_problem = oad.optimize_problem(CONFIGURATION_FILE, overwrite=True)  # Run performance analysis (no design optimization)

The output file can be visualized with the variable viewer.

In [ ]:
PERFO_OUTPUT_FILE = pth.join(WORK_FOLDER_PATH, 'operational_mission_out.xml')
oad.variable_viewer(PERFO_OUTPUT_FILE)

In [ ]:
# Energy breakdown
from utils.postprocessing.analysis_and_plots import *
fig = energy_breakdown_sun_plot_drone(PERFO_OUTPUT_FILE, mission_name='concordia_study')
fig.show()

## 2. Design of Experiments

We are now focusing on the uncertainty analysis. In contrast with the single performance evaluation ran above, the uncertainty analysis relies on multiple evaluations of the UAV model to analyse the effects of varying one or several parameters on the output variables of interest.

The OpenTurns and SALib libraries are used to set up the DoEs and run sensitivity analyses: 
- https://openturns.github.io/openturns/latest/index.html
- https://salib.readthedocs.io/en/latest/index.html

The OpenMDAO drivers are provided by Onera's OpenMDAO Extensions: https://github.com/OneraHub/openmdao_extensions

In [ ]:
from utils.postprocessing.monte_carlo import *
from utils.postprocessing.sobol import *
from utils.postprocessing.morris import *

## 2.1 Screening procedure - Morris method

The Morris method allows to get a measure of importance and interaction of input factors. It is used as a screening method to reduce the number of parameters prior to a more detailed analysis.

References

- Ruano, M.V., Ribes, J., Seco, A., Ferrer, J., 2012. An improved sampling strategy based on trajectory design for application of the Morris method to systems with many input factors. Environmental Modelling & Software 37, 103–109. https://doi.org/10.1016/j.envsoft.2012.03.008
- Morris, M.D., 1991. Factorial Sampling Plans for Preliminary Computational Experiments. Technometrics 33, 161–174. https://doi.org/10.1080/00401706.1991.10484804
- Campolongo, F., Cariboni, J., Saltelli, A., 2007. An effective screening design for sensitivity analysis of large models. Environmental Modelling & Software, Modelling, computer-assisted simulations, and mapping of dangerous phenomena for hazard assessment 22, 1509–1518. https://doi.org/10.1016/j.envsoft.2006.10.004


In [ ]:
morris_analysis(CONFIGURATION_FILE, PERFO_OUTPUT_FILE)

## 2.2 Global sensitivity analysis - Sobol' indices using Saltelli's sampling scheme

The Sobol' method allows to get a representation of the contribution of the inputs to the overall uncertainty in the model output. The space of the uncertain inputs is explored with a Monte Carlo sampling (here Saltelli's scheme, which extends the Sobol' sequence in a way to reduce the error rates in the resulting sensitivity index calculations).


References

- Campolongo, F., Saltelli, A., Cariboni, J., 2011. From screening to quantitative sensitivity analysis. A unified approach. Computer Physics Communications 182, 978–988. https://doi.org/10.1016/j.cpc.2010.12.039
- Saltelli, A., 2002. Making best use of model evaluations to compute sensitivity indices. Computer Physics Communications 145, 280–297. https://doi.org/10.1016/S0010-4655(02)00280-1
- Sobol’, I.M., 2001. Global sensitivity indices for nonlinear mathematical models and their Monte Carlo estimates. Mathematics and Computers in Simulation, The Second IMACS Seminar on Monte Carlo Methods 55, 271–280. https://doi.org/10.1016/S0378-4754(00)00270-6


In [ ]:
sobol_analysis(CONFIGURATION_FILE, PERFO_OUTPUT_FILE)

## 2.3 Distribution analysis with a regular Monte Carlo approach

Running model on DoE cases using OpenTURNS sampling methods. This regular Monte Carlo simulation is faster than the Sobol' method, but provides less information on the variance decomposition.

In [ ]:
montecarlo_siso(CONFIGURATION_FILE, PERFO_OUTPUT_FILE)

In [ ]:
montecarlo_miso(CONFIGURATION_FILE, PERFO_OUTPUT_FILE)